<a href="https://colab.research.google.com/github/agroimpacts/nmeo/blob/class%2Ff2023/materials/code/notebooks/drone_image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stacking and comparing drone orthomosaicks

In this exercise, we will process the orthomosaicks created in PIX4D into multiband, cloud-optimized geotiffs, and then compare the differences in their values to evaluate the impact of 1) PPK correction, and 2) the choice to approach used to calculate surface reflectance

In [ ]:
from google.colab import drive
root = '/content/gdrive'
drive.mount(root)

## Installs and imports

In [ ]:
%%capture
# %pip install affine
%pip install leafmap
%pip install localtileserver
%pip install rio-cogeo
%pip install rioxarray

Restart runtime and then run imports

In [ ]:
import os
import re
import sys
from pathlib import Path
from subprocess import run
import pandas as pd
import numpy as np
import leafmap.leafmap as leafmap
import localtileserver
import geopandas as gpd
import rasterio
from rasterio.plot import show
import rioxarray as rxr
import xarray as xr

## Process images

### Paths

In [ ]:
root = '/content/gdrive/MyDrive'
img_dir = Path(root) / "data/nmeo/drones/p4d/"
os.listdir(img_dir)

### List files and directories

In [ ]:
patterns = ("green.tif", "red.tif", "edge.tif", "nir.tif")
paths = []
for path, subdirs, files in os.walk(img_dir):
    for file in files:
        # print(file)
        if 'transparent_refl' in file and file.endswith(patterns):
            paths.append(str(Path(path) / file))

Note you will have to change to code below to make the paths and folder name conventions match the ones you are using

In [ ]:
path_df = pd.DataFrame({
    "type": [re.sub("0054_", "", path.split("/")[8]) for path in paths],
    "path": paths
})
path_df

### Stack PPK-corrected images

Read in each band of the PPK corrected image, mask the nodata areas, reproject, and clip to an extent

In [ ]:
imgs = []
for i, row in path_df.query("type=='ppk'").iterrows():
    img = rxr.open_rasterio(row.path).squeeze().copy()
    nodat = img.rio.nodata

    # Mask out no data areas
    img = img.where(img != nodat)
    # img.rio.write_nodata(nodat, encoded=True, inplace=True)

    # increment bands and append to list
    img["band"] = i+1
    imgs.append(img)

Stack reproject, and write to disk

In [ ]:
img_stack = xr.concat(imgs, dim="band")
img_stack = img_stack.rio.reproject("EPSG:4326", resolution=0.11/111111)\
    .rio.clip_box(minx=-71.8101, miny=42.1193,
                  maxx=-71.8041, maxy=42.1242)
img_stack.rio.write_nodata(np.nan, encoded=True, inplace=True)

out_path = Path(img_dir) / "ortho_ppk_stack.tif"
img_stack.rio.to_raster(out_path)

COG-ify

In [ ]:
cmd = ['rio', 'cogeo', 'create', '-b', '1,2,3,4', out_path, out_path]
p = run(cmd, capture_output=True)
msg = p.stderr.decode().split('\n')
print(f'...{msg[-2]}')

cmd = ['rio', 'cogeo', 'validate', out_path]
p = run(cmd, capture_output = True)
msg = p.stdout.decode().split('\n')
print(f'...{msg[0]}')

### Stack non-PPK images

Read in each band of the non-PPK corrected images, mask the nodata areas

In [ ]:
imgs = []
for i, row in path_df.query("type=='noppk'").iterrows():
    img = rxr.open_rasterio(row.path).squeeze()
    nodat = img.rio.nodata

    # Mask out no data areas
    img = img.where(img != nodat)
    # img.rio.write_nodata(nodat, encoded=True, inplace=True)

    # increment bands and append to list
    img["band"] = i+1
    imgs.append(img)

Stack and write to disk. Note that here we use `rio.reproject_match` to align the img_stack of noppk to the ppk raster. This allows us to do band math between the two images.

In [ ]:
img_stack = xr.concat(imgs, dim="band")
ppk = rxr.open_rasterio(str(Path(img_dir) / "ortho_ppk_stack.tif"))

img_stack = img_stack.rio.reproject_match(ppk)
img_stack.rio.write_nodata(np.nan, encoded=True, inplace=True)

out_path = Path(img_dir) / "ortho_noppk_stack.tif"
img_stack.rio.to_raster(out_path)

COG-ify

In [ ]:
cmd = ['rio', 'cogeo', 'create', '-b', '1,2,3,4', out_path, out_path]
p = run(cmd, capture_output=True)
msg = p.stderr.decode().split('\n')
print(f'...{msg[-2]}')

cmd = ['rio', 'cogeo', 'validate', out_path]
p = run(cmd, capture_output = True)
msg = p.stdout.decode().split('\n')
print(f'...{msg[0]}')

### Plot and compare

## Calculate differences in images

In [ ]:
cog_paths = [
    str(Path(img_dir) / "ortho_ppk_stack.tif"),
    str(Path(img_dir) / "ortho_noppk_stack.tif")
]

ppk = rxr.open_rasterio(cog_paths[0])#.squeeze()
noppk = rxr.open_rasterio(cog_paths[1])#.squeeze()

In [ ]:
ppk_noppk_dif = (ppk - noppk)
out_path = Path(img_dir) / "ortho_ppk-noppk_diff.tif"
ppk_noppk_dif.rio.to_raster(out_path)
# ppk_noppk_dif.plot.imshow(col="band", col_wrap=2, cmap="Greys_r")

Visualize

In [ ]:
m = leafmap.Map()
m.add_basemap()
m.add_basemap("SATELLITE")
m.add_raster(str(out_path), bands=[1, 2, 3], layer_name="PPK")
m

## Homework

- Use pix4d together with the PPK corrected images from eMotion3 to make orthomosaicks that do not use the calibration targets (you can either reset the calibration targets or load the list images into a new pix4d project that excludes those images with targets in them)
- Adapt this notebook to process the orthomosaicks into a cropped, reprojected multiband tiff that aligns with the PPK raster.
- Compare differences between the PPK images and the new one made using a different reflectance conversion.  